<a href="https://colab.research.google.com/github/LucasCPimentel/dialogflow-finance-chatbot/blob/main/ProjetoETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto ETL - Desafio Santander Bootcamp 2023 em Ciência de Dados / Python


## Objetivo:

### Este projeto consiste na criação de um sistema de geração de mensagens personalizadas para clientes bancários, utilizando o Dialogflow da Google e integração com uma API bancária fictícia. As principais tecnologias empregadas incluem Python, Pandas para manipulação de dados, Dialogflow para processamento de linguagem natural e interações de chat, além de requisições HTTP para acessar informações dos clientes na API bancária. O sistema utiliza Inteligência Artificial para criar mensagens customizadas sobre a importância dos investimentos para cada cliente, contribuindo para uma experiência mais relevante e engajadora.




## **0.0. Imports**

In [ ]:
!pip install pandas
!pip install requests
!pip install numpy
!pip install matplotlib
!pip install openai
!pip install google-cloud-dialogflow

In [7]:
import pandas as pd
import requests
import json
import numpy as np
import os
import matplotlib.pyplot as plt
from google.cloud import dialogflow

## **0.1. Definindo as informações necessárias**

In [8]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'
dialogflow_project_id = 'projetoetldialogflow'

## **0.2. Configurando as credenciais**

In [19]:
json_credentials_path = "/content/projetoetldialogflow-a1f40040a018.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_credentials_path


## **0.3. Carregando dados**

In [10]:
df = pd.read_csv('SDW20233.csv.txt')
user_ids = df['UserID'].tolist()

## **0.4. Função para obter informações do usuário**

In [11]:
def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

## **0.5. Coletar informações do usuário**

In [12]:
users = [user for id in user_ids if (user := get_user(id)) is not None]

## **0.6. Função para gerar frases sobre a importância de investir usando o DialogFlow**

In [34]:
def generate_dialogflow_news(user):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(dialogflow_project_id, user['id'])

    text_input = dialogflow.TextInput(
        text=f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)",
        language_code="pt-BR"
    )
    query_input = dialogflow.QueryInput(text=text_input)
    response = session_client.detect_intent(
        request={"session": session, "query_input": query_input}
    )

    return response.query_result.fulfillment_text

## **0.7. Adição de frases geradas aos usuários**

In [40]:
for user in users:
    news = generate_dialogflow_news(user)
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })
    print(f"{user['name'] }, {news}")

João, Investir é o caminho para conquistar sonhos.
Maria, Prepare-se para o futuro, comece a investir.
André, Investir é o caminho para conquistar sonhos.
Laura, Seu futuro agradece: invista para prosperar.
Lucas, Transforme pequenos investimentos em grandes conquistas.
Ana, Faça seu dinheiro trabalhar por você. Invista!
Pedro, Faça seu dinheiro trabalhar por você. Invista!
Sofia, Invista hoje e colha os frutos amanhã!
Diego, Invista para garantir um futuro financeiramente estável.
Camila, Invista com sabedoria e veja seu patrimônio crescer.


## **0.8. Função para atualizar os dados do usuário**

In [22]:
def update_user(user):
    response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

## **0.9. Atualizar usuários no sistema**

In [23]:
for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")

User João updated? True!
User Maria updated? True!
User André updated? True!
User Laura updated? True!
User Lucas updated? True!
User Ana updated? True!
User Pedro updated? True!
User Sofia updated? True!
User Diego updated? True!
User Camila updated? True!


# **Análise das Contas Bancárias e Limites de Crédito**

## **0.10. Lista para armazenar os saldos das contas bancárias e os limites de crédito**

In [24]:
account_balances = []
credit_limits = []

## **0.11. Coletar os saldos das contas bancárias e limites de crédito**

In [25]:
for user in users:
    account_balances.append(user['account']['balance'])
    credit_limits.append(user['account']['limit'])
    credit_limits.append(user['card']['limit'])

## **0.12. Estatística dos saldos das contas bancárias**

In [26]:
average_balance = np.mean(account_balances)
median_balance = np.median(account_balances)
std_dev_balance = np.std(account_balances)

## **0.13. Estatística dos limites de crédito**

In [27]:
average_limit = np.mean(credit_limits)
median_limit = np.median(credit_limits)
std_dev_limit = np.std(credit_limits)

## **0.14. Estatísticas dos saldos das contas bancárias e limites de crédito**

In [28]:
print("Análise das Contas Bancárias:")
print(f"Média dos saldos das contas: {average_balance}")
print(f"Mediana dos saldos das contas: {median_balance}")
print(f"Desvio padrão dos saldos das contas: {std_dev_balance}")

print("\nAnálise dos Limites de Crédito:")
print(f"Média dos limites de crédito: {average_limit}")
print(f"Mediana dos limites de crédito: {median_limit}")
print(f"Desvio padrão dos limites de crédito: {std_dev_limit}")

Análise das Contas Bancárias:
Média dos saldos das contas: 2900.0
Mediana dos saldos das contas: 2250.0
Desvio padrão dos saldos das contas: 2273.3235581412514

Análise dos Limites de Crédito:
Média dos limites de crédito: 3440.0
Mediana dos limites de crédito: 3000.0
Desvio padrão dos limites de crédito: 2387.1321706181247


In [29]:
!git clone https://github.com/LucasCPimentel/dialogflow-finance-chatbot.git

Cloning into 'dialogflow-finance-chatbot'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), done.


In [41]:
!mv /content/dialogflow-finance-chatbot/* /content/dialogflow-finance-chatbot/


mv: '/content/dialogflow-finance-chatbot/README.md' and '/content/dialogflow-finance-chatbot/README.md' are the same file


In [42]:
cd /ccontent/dialogflow-finance-chatbot/

[Errno 2] No such file or directory: '/ccontent/dialogflow-finance-chatbot/'
/content
